# Transformaciones

A lo largo de este archivo se modificara los datos con el fin de poder acceder mas facilmente a esto dentro de PowerBI

Primero importamos las librerias

In [1]:
import pandas as pd



In [2]:
# Cargar las hojas del archivo de Excel en dataframes por separado
hechos_df = pd.read_excel(r"C:\Users\mateo\OneDrive\Escritorio\data science\labs 2\SINIESTROSVIALES\Datos\homicidios.xlsx", sheet_name='HECHOS')
victimas_df = pd.read_excel(r"C:\Users\mateo\OneDrive\Escritorio\data science\labs 2\SINIESTROSVIALES\Datos\homicidios.xlsx", sheet_name='VICTIMAS')

In [3]:
print("Primeras dos filas del dataframe HECHOS:")
display(hechos_df.head(2))



Primeras dos filas del dataframe HECHOS:


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


In [4]:
print("\nTipos de datos de las columnas en HECHOS:")
print(hechos_df.dtypes)


Tipos de datos de las columnas en HECHOS:
ID                               object
N_VICTIMAS                        int64
FECHA                    datetime64[ns]
AAAA                              int64
MM                                int64
DD                                int64
HORA                             object
HH                               object
LUGAR_DEL_HECHO                  object
TIPO_DE_CALLE                    object
Calle                            object
Altura                          float64
Cruce                            object
Dirección Normalizada            object
COMUNA                            int64
XY (CABA)                        object
pos x                            object
pos y                            object
PARTICIPANTES                    object
VICTIMA                          object
ACUSADO                          object
dtype: object


In [5]:
print("\nPrimeras dos filas del dataframe VICTIMAS:")
display(victimas_df.head(2))


Primeras dos filas del dataframe VICTIMAS:


,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


In [6]:
print("\nTipos de datos de las columnas en VICTIMAS:")
print(victimas_df.dtypes)


Tipos de datos de las columnas en VICTIMAS:
ID_hecho                       object
FECHA                  datetime64[ns]
AAAA                            int64
MM                              int64
DD                              int64
ROL                            object
VICTIMA                        object
SEXO                           object
EDAD                           object
FECHA_FALLECIMIENTO            object
dtype: object


## Transformaciones de hechos

In [7]:
# Crear la columna Semestre
hechos_df['Semestre'] = hechos_df.apply(lambda row: f"{row['AAAA']}-1" if row['MM'] <= 6 else f"{row['AAAA']}-2", axis=1)



In [8]:
# Crear la columna Poblacion de Buenos Aires
hechos_df['Poblacion de Buenos Aires'] = 31217107



In [9]:
# Crear la columna Tasa de homicidios
hechos_df['Tasa de homicidios'] = hechos_df.groupby('Semestre')['N_VICTIMAS'].transform('sum')



In [10]:
# Crear la columna "tasa homicidios semestre anterior"
def calcular_tasa_semestre_anterior(row, df):
    semestre_actual = row['Semestre']
    año_actual = int(semestre_actual.split('-')[0])
    semestre_numero = int(semestre_actual.split('-')[1])
    
    if semestre_numero == 1:
        semestre_anterior = f"{año_actual - 1}-2"
    else:
        semestre_anterior = f"{año_actual}-1"
    
    tasa_anterior = df[df['Semestre'] == semestre_anterior]['Tasa de homicidios']
    return tasa_anterior.iloc[0] if not tasa_anterior.empty else 0

hechos_df['Tasa Homicidios Semestre Anterior'] = hechos_df.apply(lambda row: calcular_tasa_semestre_anterior(row, hechos_df), axis=1)


In [11]:
# Función para calcular KPI1
def calcular_kpi1(df):
    kpi1 = []
    for index, row in df.iterrows():
        current_semester = row['Semestre']
        current_year = int(current_semester.split('-')[0])
        current_half = int(current_semester.split('-')[1])
        
        # Semestre anterior
        if current_half == 1:
            previous_semester = f"{current_year-1}-2"
        else:
            previous_semester = f"{current_year}-1"
        
        # Tasa de homicidios actual y anterior
        current_rate = (df[df['Semestre'] == current_semester]['N_VICTIMAS'].sum() / 31217107) * 100000
        previous_rate = (df[df['Semestre'] == previous_semester]['N_VICTIMAS'].sum() / 31217107) * 100000
        
        if previous_rate == 0:
            kpi1.append('SD')
        else:
            kpi1_value = ((previous_rate - current_rate) / previous_rate) * 100
            kpi1.append(kpi1_value)
    
    return kpi1

# Crear la columna KPI1
hechos_df['KPI1'] = calcular_kpi1(hechos_df)



In [12]:
# Crear la columna "victimas moto"
hechos_df['Victimas Moto'] = hechos_df.apply(lambda row: row['N_VICTIMAS'] if row['VICTIMA'] == 'MOTO' else 0, axis=1)


In [13]:
# Crear la columna "victimas moto año pasado"
def calcular_victimas_moto_anterior(row, df):
    año_actual = row['AAAA']
    año_anterior = año_actual - 1
    victimas_anterior = df[(df['AAAA'] == año_anterior) & (df['VICTIMA'] == 'MOTO')]['N_VICTIMAS'].sum()
    return victimas_anterior if victimas_anterior else 0

hechos_df['Victimas Moto Año Pasado'] = hechos_df.apply(lambda row: calcular_victimas_moto_anterior(row, hechos_df), axis=1)


In [14]:
# Función para calcular KPI2
def calcular_kpi2(df):
    kpi2 = []
    for index, row in df.iterrows():
        current_year = row['AAAA']
        
        # Año anterior
        previous_year = current_year - 1
        
        # Número de accidentes mortales con víctimas en moto
        current_motorcycle_accidents = df[(df['AAAA'] == current_year) & (df['VICTIMA'] == 'MOTO')].shape[0]
        previous_motorcycle_accidents = df[(df['AAAA'] == previous_year) & (df['VICTIMA'] == 'MOTO')].shape[0]
        
        if previous_motorcycle_accidents == 0:
            kpi2.append('SD')
        else:
            kpi2_value = ((previous_motorcycle_accidents - current_motorcycle_accidents) / previous_motorcycle_accidents) * 100
            kpi2.append(kpi2_value)
    
    return kpi2

# Crear la columna KPI2
hechos_df['KPI2'] = calcular_kpi2(hechos_df)



In [15]:
# Mostrar las primeras dos filas del dataframe actualizado
hechos_df.head(2)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,VICTIMA,ACUSADO,Semestre,Poblacion de Buenos Aires,Tasa de homicidios,Tasa Homicidios Semestre Anterior,KPI1,Victimas Moto,Victimas Moto Año Pasado,KPI2
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,MOTO,AUTO,2016-1,31217107,65,0,SD,1,0,SD
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,AUTO,PASAJEROS,2016-1,31217107,65,0,SD,0,0,SD


## Transformaciones de Victimas

In [16]:
#Crear una columna que diga los menores que son victimas anuales
victimas_df['EDAD'] = pd.to_numeric(victimas_df['EDAD'], errors='coerce')
victimas_df['Menoresvictimasanual'] = victimas_df.apply(lambda row: 1 if 0 <= row['EDAD'] <= 17 else 0, axis=1)
menores_victimas_anuales = victimas_df.groupby('AAAA')['Menoresvictimasanual'].sum().reset_index()

# Merge con el dataframe original para incluir la columna Menoresvictimasanual
victimas_df = victimas_df.merge(menores_victimas_anuales, on='AAAA', suffixes=('', '_total'))


In [17]:
def calcular_menores_victimas_anterior(row, df):
    año_actual = row['AAAA']
    año_anterior = año_actual - 1
    menores_anterior = df[(df['AAAA'] == año_anterior) & (df['EDAD'] <= 17)].shape[0]
    return menores_anterior

victimas_df['Menoresvictimasanual_Ant'] = victimas_df.apply(lambda row: calcular_menores_victimas_anterior(row, victimas_df), axis=1)


In [18]:
def calcular_kpi3(df):
    kpi3 = []
    for index, row in df.iterrows():
        current_year = row['AAAA']
        previous_year = current_year - 1

        current_year_victims = df[df['AAAA'] == current_year]['Menoresvictimasanual_total'].iloc[0]
        previous_year_victims = df[df['AAAA'] == previous_year]['Menoresvictimasanual_total'].iloc[0] if previous_year in df['AAAA'].values else 0

        if previous_year_victims == 0:
            kpi3.append('SD')
        else:
            kpi3_value = ((current_year_victims - previous_year_victims) / previous_year_victims) * 100
            kpi3.append(kpi3_value)

    return kpi3

# Crear la columna KPI3
victimas_df['KPI3'] = calcular_kpi3(victimas_df)


In [19]:
victimas_df.head(2)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Menoresvictimasanual,Menoresvictimasanual_total,Menoresvictimasanual_Ant,KPI3
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01 00:00:00,0,3,0,SD
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02 00:00:00,0,3,0,SD


Se guardan los DF

In [20]:
hechos_df.to_excel('hechos_actualizado.xlsx', index=False)
victimas_df.to_excel('victimas_actualizado.xlsx', index=False)
